In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd
import pyLDAvis.gensim_models

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm.notebook import tqdm
import html

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
mastodon_data = pd.read_feather(
    "../1_data_collection/data/8_account_general_data.feather"
)
docs = mastodon_data["account_descri"]

In [ ]:
translator = GoogleTranslator(source="auto", target="en")

In [ ]:
for i, row in tqdm(mastodon_data.iterrows(), total=len(mastodon_data)):
    mastodon_data.loc[i, "trl_account_descri"] = translator.translate(
        row["account_descri"]
    )

In [ ]:
mastodon_data.loc[mastodon_data["language_type"] != "English", "trl_account_descri"]

In [ ]:
mastodon_data["trl_account_descri"] = mastodon_data["trl_account_descri"].apply(
    lambda x: html.unescape(x) if x is not None else x
)

In [ ]:
mastodon_data.to_feather("./data/2_account_descri_translated.feather")

In [ ]:
mastodon_data = pd.read_feather("./data/2_account_descri_translated.feather")

In [ ]:
mastodon_data_w_acct_desc = mastodon_data[~mastodon_data["trl_account_descri"].isna()]

In [ ]:
from gensim import corpora


from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Sample documents
documents = mastodon_data_w_acct_desc["trl_account_descri"]

# Tokenize and preprocess the documents
stop_words = set(stopwords.words("english"))
documents = [word_tokenize(doc.lower()) for doc in documents]
documents = [
    [word for word in doc if word.isalnum() and word not in stop_words]
    for doc in documents
]

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(documents)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train the LDA model
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=15)

# Print the topics and their top words
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.save_html(vis, "./data/acct_descr_topic.html")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

fuLL_acct_str = " ".join([" ".join(doc) for doc in documents])

wc = WordCloud(
    width=400, height=400, background_color="white", repeat=True, max_words=35
)
wc.generate(fuLL_acct_str)

fig, ax = plt.subplots(figsize=(5, 5))
plt.axis("off")
ax.imshow(wc, interpolation="bilinear")
plt.show()

In [ ]:
import pprint

pprint.pprint(topics)

In [ ]:
acct_desc_topics = []
for doc in documents:
    acct_doc_bow = dictionary.doc2bow(doc)
    predicted_topic_probs = lda_model.get_document_topics(acct_doc_bow)

    acct_topic = None
    max_prob = 0
    for topic_prob in predicted_topic_probs:
        check_topic = topic_prob[0]
        check_prob = topic_prob[1]
        if check_prob > max_prob:
            max_prob = check_prob
            acct_topic = check_topic
    acct_desc_topics.append(acct_topic)

mastodon_data_w_acct_desc["acct_descri_topic"] = acct_desc_topics

In [ ]:
final_mastodon_data = pd.concat(
    [
        mastodon_data_w_acct_desc,
        mastodon_data[mastodon_data["trl_account_descri"].isna()],
    ]
).reset_index(drop=True)

In [ ]:
final_mastodon_data.to_feather("./data/3_acct_descr_topics.feather")